## Cross-Platform verification

In this tutorial, we will show to measure the fidelity $\mathcal{F}=\mathrm{Tr}(\rho_1\rho_2)/\sqrt{\mathrm{Tr}(\rho_1^2)\mathrm{Tr}(\rho_2^2)}$ between two states $\rho_1$, $\rho_2$ prepared on two quantum devices. 

Related papers [Elben et al, PRL 2019](https://doi.org/10.1103/PhysRevLett.124.010504), [Zhu et al, Nat. Comm 2022](https://www.nature.com/articles/s41467-022-34279-5)

In [1]:
using Revise
using RandomMeas
using ProgressMeter

We first consider a state $\rho_1$ prepared by a random quantum circuit

In [3]:
N = 4
ξ = siteinds("Qubit", N)
depth = 2
circuit = random_circuit(ξ, depth)
states = ["Dn" for n in 1:N]
ψ0 = MPS(ξ,states);
ρ0 = outer(ψ0',ψ0);

# Prepare state 1
ρ1 = apply(circuit,ρ0,apply_dag=true);
println("State 1 prepared")

# Prepare state 2
p = 0.1*rand(N)
ρ2 = apply_depo_channel(ρ1,p)
println("State 2 prepared")


State 1 prepared
State 2 prepared


In [8]:
overlap = real(inner(ρ1,ρ2))
purity1 = get_trace_moment(ρ1,2)
purity2 = get_trace_moment(ρ2,2)
F = overlap/sqrt(purity1*purity2)
println("Overlap ",overlap)
println("Purity1 ",purity1)
println("Purity2 ",purity2)
println("Fidelity ",F)

Overlap 0.8271558797915574
Purity1 1.0000000000000022
Purity2 0.690324201944853
Fidelity 0.9955448074107242


We simulate the experiment by sampling randomized measurement on state ρ1 and ρ2. In our simulated experiment, the second state $\rho_2$ differs from $\rho_1$ due to local depolarization. We sample randomized measurements according to the same random unitaries $u$. For some reason, experiment 2 is a bit faster than experiment 1. Thus, we choose a higher number of measurements per unitary, $N_M = 500$.

In [6]:
NU,NM1,NM2 = 500,100,200
measurements1 = Vector{MeasurementData{LocalUnitaryMeasurementSetting}}(undef,NU)
measurements2 = Vector{MeasurementData{LocalUnitaryMeasurementSetting}}(undef,NU)
@showprogress dt=1 for r in 1:NU
    measurement_setting = LocalUnitaryMeasurementSetting(N; site_indices=ξ,ensemble="Haar")
    measurements1[r] = MeasurementData(ρ1,NM1,measurement_setting;mode="dense")
    measurements2[r] = MeasurementData(ρ2,NM2,measurement_setting;mode="dense")
end
measurement_group1= MeasurementGroup(measurements1)
measurement_group2= MeasurementGroup(measurements2);

Progress: 100%|█████████████████████████████████████████| Time: 0:00:02


We then use the formula $\mathrm{tr}(\rho_1\rho_2)=(-2)^{-D[s,s']}\sum_{s,s'}P_u^{(1)}(s)P_u^{(2)}(s')$ to extract the overlap (and proceed similarly to access the purities $\mathrm{tr}(\rho_1^2)$, $\mathrm{tr}(\rho_2^2)$). In this process, the Born probabilities for each random unitary are computed. Thus, the process is not measurment efficient.

In [9]:
overlap = get_overlap(measurement_group1,measurement_group2)
println("Estimated overlap = ",overlap)
purity_1 = get_purity(measurement_group1)
println("Estimated purity 1 = ",purity_1)
purity_2 = get_purity(measurement_group2)
println("Estimated purity 2 = ",purity_2)
Fs= get_fidelity(measurement_group1,measurement_group2)
println("Estimated fidelity ",Fs)

Estimated overlap = 0.8126917000000005
Estimated purity 1 = 0.972086060606061
Estimated purity 2 = 0.6806007035175878
Estimated fidelity 0.9991421250226076
